In [145]:
# import all prerequisites
from PyFoam.Applications.CloneCase import CloneCase
from PyFoam.Applications.ClearCase import ClearCase
from PyFoam.Applications.CreateBoundaryPatches import CreateBoundaryPatches
from PyFoam.Applications.ChangeBoundaryName import ChangeBoundaryName
from PyFoam.Applications.ChangeBoundaryType import ChangeBoundaryType
from PyFoam.Applications.WriteDictionary import WriteDictionary
from PyFoam.Applications.Runner import Runner
from PyFoam.Applications.MeshUtilityRunner import MeshUtilityRunner
import shutil, os, sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
sandia_path = "/home/pavel.kholodov/WORK/OpenFOAM/Benchmark/combustion/SandiaFlameD/flame/CaseB/B2_1X"
newcase = "Mixing"
meshfile = "Chamber_2D_axi.msh"

In [ ]:
%%capture capt
# Close case from existing one
CloneCase(args=["--no-vcs",sandia_path,newcase,"--no-pyfoam","--add-item=0.orig","--force"])

In [ ]:
%%capture capt
# Clear case
#ClearCase(args=["-h"])
ClearCase(args=["--remove-analyzed","--no-pyfoam","--additional=Mixing.foam","--additional=0",newcase])

In [ ]:
%%capture capt
Runner(args=["--silent","fluentMeshToFoam","-case",os.path.join(os.getcwd(),newcase),meshfile])

In [ ]:
%%capture capt
#shutil.copytree("./Mixing/0.orig","./Mixing/0")
ClearCase(args=["--remove-analyzed","--no-pyfoam","--additional=Mixing.foam",newcase])

In [ ]:
%%capture capt
#Change names
ChangeBoundaryName(args=[newcase,"OX_INLET","ox_inlet"])
ChangeBoundaryName(args=[newcase,"FUEL_INLET","fuel_inlet"])
ChangeBoundaryName(args=[newcase,"BOTTOM","bottom"])
ChangeBoundaryName(args=[newcase,"CHAMBER_WALL","chamber_wall"])
ChangeBoundaryName(args=[newcase,"NOZZLE_WALL","nozzle_wall"])
ChangeBoundaryName(args=[newcase,"JET_WALL","jet_wall"])
ChangeBoundaryName(args=[newcase,"COFLOW","coflow"])
ChangeBoundaryName(args=[newcase,"ATMOSPHERE","atmosphere"])
ChangeBoundaryName(args=[newcase,"OUTLET","outlet"])
ChangeBoundaryName(args=[newcase,"FRONT","front"])
ChangeBoundaryName(args=[newcase,"BACK","back"])

bc_dict = {'fuel_inlet':'fuel_inlet','oxidizer_inlet':'ox_inlet','outlet':'outlet','opening':'atmosphere','coflow':'coflow'}

In [ ]:
%%capture capt
# Change boundary type to appropriate
ChangeBoundaryType(args=[newcase,"ox_inlet","patch"])
ChangeBoundaryType(args=[newcase,"fuel_inlet","patch"])
ChangeBoundaryType(args=[newcase,"coflow","patch"])
ChangeBoundaryType(args=[newcase,"atmosphere","patch"])
ChangeBoundaryType(args=[newcase,"outlet","patch"])
ChangeBoundaryType(args=[newcase,"front","wedge"])
ChangeBoundaryType(args=[newcase,"back","wedge"])

In [ ]:
%%capture capt
# Delete unneeded BC files
for i in ['CH4','OH','G','H2O','CO2','CO','O','H']:
    os.remove("./"+newcase+"/0/"+i)

In [ ]:
%%capture capt
# Clear all boundary conditions from old names
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/U")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/p")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/T")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/k")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/epsilon")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/nut")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/alphat")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/O2")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/H2")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/N2")])

In [ ]:
%%capture capt
# BC for velocity
#Wall BC
CreateBoundaryPatches(args=["--overwrite","--default={'type':'fixedValue','value':'uniform (0 0 0)'}",os.path.join(os.getcwd(),newcase,"0/U")])
# Inlet BC
for bcname in [bc_dict['oxidizer_inlet'],bc_dict['fuel_inlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,"--default={'type':'flowRateInletVelocity'}",os.path.join(os.getcwd(),newcase,"0/U")])
# Outlet BC
CreateBoundaryPatches(args=["--overwrite","--filter=coflow","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/U")])
CreateBoundaryPatches(args=["--overwrite","--filter=outlet","--default={'type':'pressureInletOutletVelocity','value':'uniform (0 0 0)'}",os.path.join(os.getcwd(),newcase,"0/U")])
CreateBoundaryPatches(args=["--overwrite","--filter=outlet","--default={'type':'pressureInletOutletVelocity','value':'uniform (0 0 0)'}",os.path.join(os.getcwd(),newcase,"0/U")])
# Wedges
CreateBoundaryPatches(args=["--overwrite","--filter=front","--default={'type':'wedge'}",os.path.join(os.getcwd(),newcase,"0/U")])
CreateBoundaryPatches(args=["--overwrite","--filter=back","--default={'type':'wedge'}",os.path.join(os.getcwd(),newcase,"0/U")])

In [ ]:
%%capture capt
# BC for pressure
# Wall BC
CreateBoundaryPatches(args=["--overwrite","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/p")])
# Inlet BC
CreateBoundaryPatches(args=["--overwrite","--filter=ox_inlet","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/p")])
#CreateBoundaryPatches(args=["--overwrite","--filter=fuel_inlet","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/U")])
# Outlet BC
CreateBoundaryPatches(args=["--overwrite","--filter=outlet","--default={'type':'pressureInletOutletVelocity','value':'uniform 98066.5'}",os.path.join(os.getcwd(),newcase,"0/p")])
CreateBoundaryPatches(args=["--overwrite","--filter=coflow","--default={'type':'pressureInletOutletVelocity','value':'uniform 98066.5'}",os.path.join(os.getcwd(),newcase,"0/p")])

In [ ]:
%%capture capt
# BC for H2,O2,N2 components
files = ['O2','H2','N2']
# Wall BC
for i in files:
    CreateBoundaryPatches(args=["--overwrite","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0",i)])
# Inlet BC
# oxidizer inlet
values = ['1.0','0.0','0.0']
for i in range(len(files)):
    CreateBoundaryPatches(args=["--overwrite","--filter="+bc_dict['oxidizer_inlet'],\
                                "--default={'type':'fixedValue','value':'uniform %s'}"%(values[i]),\
                                os.path.join(os.getcwd(),newcase,"0",files[i])])
# fuel inlet
values = ['0.0','1.0','0.0']
    CreateBoundaryPatches(args=["--overwrite","--filter="+bc_dict['fuel_inlet'],\
                                "--default={'type':'fixedValue','value':'uniform %s'}"%(values[i]),\
                                os.path.join(os.getcwd(),newcase,"0",files[i])])
# Outlet BC - Atmosphere
files = ['O2','H2','N2']
values = ['0.23','0.0','0.77']
for i in range(len(files)):
    CreateBoundaryPatches(args=["--overwrite","--filter="+bc_dict['opening'],\
                                "--default={'type':'fixedValue','value':'uniform %s'}"%(values[i]),\
                                os.path.join(os.getcwd(),newcase,"0",files[i])])
    CreateBoundaryPatches(args=["--overwrite","--filter="+bc_dict['outlet'],\
                                "--default={'type':'fixedValue','value':'uniform %s'}"%(values[i]),\
                                os.path.join(os.getcwd(),newcase,"0",files[i])])

# BC for Ydefault components
CreateBoundaryPatches(args=["--overwrite","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/Ydefault")])
for bcname in [bc_dict['oxidizer_inlet'],bc_dict['fuel_inlet'],bc_dict['opening'],bc_dict['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                                "--default={'type':'fixedValue','value':'uniform 0.0'}",\
                                os.path.join(os.getcwd(),newcase,"0/Ydefault")])


In [ ]:
%%capture capt
# BC for temperature
CreateBoundaryPatches(args=["--overwrite","--default={'type':'fixedValue','value':'uniform 293'}",os.path.join(os.getcwd(),newcase,"0/T")])

for bcname in [bc_dict['opening'],bc_dict['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,"--default={'type':'inletOutlet'}",os.path.join(os.getcwd(),newcase,"0/T")])

WriteDictionary(args=[os.path.join(os.getcwd(),newcase,"0/T"),"boundaryField['outlet']['inletValue']","293"])
WriteDictionary(args=[os.path.join(os.getcwd(),newcase,"0/T"),"boundaryField['atmosphere']['inletValue']","293"])

In [ ]:
%%capture capt
# BC for alphat
CreateBoundaryPatches(args=["--overwrite","--default={'type':'compressible::alphatWallFunction','value':'$internalField'}",os.path.join(os.getcwd(),newcase,"0/alphat")])
# BC for inlets/outlets
for bcname in [bc_dict['oxidizer_inlet'],bc_dict['fuel_inlet'],bc_dict['opening'],bc_dict['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                                "--default={'type':'calculated','value':'$internalFeld'}",\
                                os.path.join(os.getcwd(),newcase,"0/alphat")])